In [1]:
print('live')

live


In [2]:
import pandas as pd
import os
import json

pd.options.display.max_colwidth = 200

data_path = '/data_volume/core_data/'
urls_folder = '/data_volume/url_data/'
tweets_folder = '/data_volume/twitter_data/mp_tweets/'

# compile and clean text data

In [ ]:
#compile URL texts into single table

# urls = pd.read_parquet(data_path+'all_urls_no_title_or_blurb.parquet')
urls = pd.read_csv(data_path+'all_urls_no_title_or_blurb.csv')
url_rids = urls['url_rid']; del urls

url_texts = list()
for i, url_rid in enumerate(url_rids):
    subfolder = url_rid[:2]+'/'
    file = url_rid+'_parsed.txt'
    path = urls_folder+subfolder+file
    if os.path.isfile(path):
        with open(path, 'r') as f:
            text = f.read()
        url_texts.append((url_rid,text))
        
url_texts = pd.DataFrame(url_texts, columns = ['url_rid','text'])
url_texts.to_parquet('./data/url_texts.parquet')

In [ ]:
# add url_texts to URLs

url_texts = pd.read_parquet('./data/url_texts.parquet')
urls = pd.read_csv(data_path+'all_urls_no_title_or_blurb.csv')
urls = urls.drop(columns = ['Unnamed: 0'])
urls = urls.merge(url_texts, on='url_rid', how='inner')
urls['time'] = pd.to_datetime(urls['first_post_time_unix']).dt.tz_localize(None)
urls = urls.drop(columns = ['first_post_time_unix'])
urls['domain'] = urls['clean_url'].apply(lambda x: urlparse(x).netloc) #add url domain to urls
urls.to_parquet('./data/urls.parquet')

In [ ]:
#compile tweets data into single table

def extract_tweet_info(tweet_json, mp_name):
    time = tweet_json['created_at']
    text = tweet_json['text']
    tweet_id = tweet_json['id']
    user_id = tweet_json['author_id']
    n_likes = tweet_json['public_metrics']['like_count']
    n_replies = tweet_json['public_metrics']['reply_count']
    n_retweets = tweet_json['public_metrics']['retweet_count']
    n_quotes = tweet_json['public_metrics']['quote_count']
    tweet = (mp_name, tweet_id, time, text, user_id, n_likes, n_replies, n_retweets, n_quotes)
    return tweet


jsonfiles = os.listdir(tweets_folder)
tweets = list()
for file in jsonfiles:
    mp_name = file[:-5].replace('_', ' ')
    with open(tweets_folder+file, 'rb') as f:
        for line in list(f.readlines()):
            line_json = json.loads(line)
            if 'data' in line_json: #most files' lines contain a "data" field which is a list of tweets
                for tweet_json in line_json['data']:
                    tweets.append(extract_tweet_info(tweet_json, mp_name))
            elif 'text' in line_json: #some files' lines contain a tweet on each line
                tweet_json = line_json
                tweets.append(extract_tweet_info(tweet_json,mp_name))

tweets = pd.DataFrame(tweets, columns = ['mp_name', 'tweet_id', 'time', 'text', 
                                         'user_id', 'n_likes', 'n_replies', 
                                         'n_retweets', 'n_quotes'])
tweets.drop_duplicates(subset=['tweet_id','time']).reset_index(drop=True)
tweets['time'] = pd.to_datetime(tweets['time']).dt.tz_localize(None)
tweets.to_parquet('./data/mp_tweets.parquet')

In [ ]:
#clean commons speeches

commons_speeches = pd.read_csv(data_path+'commons_speeches.csv')
commons_speeches = commons_speeches.dropna(subset=['text']).reset_index(drop=True)
commons_speeches = commons_speeches[~commons_speeches['hs_8Question'].isna() | ~commons_speeches['hs_2cUrgentQuestion'].isna()]
commons_speeches = commons_speeches[['_id','py_date','text','name','party','constituency','job']]
commons_speeches = commons_speeches.rename(columns = {'_id':'commons_speech_id','py_date':'date',
                                                      'party':'constituency', 'constituency':'party',
                                                      'name':'mp_name'})
commons_speeches['date'] = pd.to_datetime(commons_speeches['date'], format = "%Y-%m-%d").dt.tz_localize(None)
commons_speeches = commons_speeches.rename(columns = {'date':'time'})
commons_speeches['mp_name'] = commons_speeches['mp_name'].apply(lambda x: x.lower() if isinstance(x,str) else None)
commons_speeches['party'] = commons_speeches['party'].replace({'Lab/ Co-op':'Lab/Co-op'})
commons_speeches = commons_speeches.reset_index(drop=True)
commons_speeches.to_parquet('./data/commons_speeches.parquet')


# clean up MP names

In [1]:
import pandas as pd
import difflib
import re

In [2]:
commons_speeches = pd.read_parquet('./data/commons_speeches.parquet')
mp_tweets = pd.read_parquet('./data/mp_tweets.parquet')

In [3]:
#drop parenthetical notes
commons_speeches['mp_name'] = commons_speeches['mp_name'].apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", x))
mp_tweets['mp_name'] = mp_tweets['mp_name'].apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", x))

#drop honorifics and standardize unusual characters to ASCII text
commons_speeches['mp_name'] = commons_speeches['mp_name'].apply(lambda x: x.replace('. ',''
                                                                            ).replace('mr ',''
                                                                            ).replace('sir ',''
                                                                            ).replace('ms ',''
                                                                            ).replace('mrs ',''
                                                                            ).replace('dame ',''
                                                                            ).replace('dr',''
                                                                            ).replace('ᶜ','c'
                                                                            ).replace(' mc ',' mc'
                                                                            ).replace('\u2002',''
                                                                            ).replace('\u2003',''
                                                                            ).replace('’',"'"
                                                                            ).strip())

mp_tweets['mp_name'] = mp_tweets['mp_name'].apply(lambda x: x.replace('. ',''
                                                                ).replace('mr ',''
                                                                ).replace('sir ',''
                                                                ).replace('ms ',''
                                                                ).replace('mrs ',''
                                                                ).replace('dame ',''
                                                                ).replace('dr',''
                                                                ).replace('ᶜ','c'
                                                                ).replace(' mc ',' mc'
                                                                ).replace('\u2002',''
                                                                ).replace('\u2003',''
                                                                ).replace('’',"'"
                                                                ).strip())

In [4]:
parties = commons_speeches['party'].unique()

In [5]:
mps = commons_speeches['mp_name'].unique()
unmatched = mp_tweets[~(mp_tweets['mp_name'].isin(mps))]['mp_name'].unique()
possible_matches = {x:difflib.get_close_matches(x, mps) for x in unmatched}
possible_matches

{'graham p jones': ['graham pjones', 'graham jones', 'sarah jones'],
 'stuart c mcdonald': ['stuart cmcdonald',
  'stewart malcolm mcdonald',
  'stuart blair donaldson'],
 'apsana begum': [],
 'paul maskey': ['paul masterton', 'paul sweeney', 'paul maynard'],
 'jonathan lord': ['jonathan edwards',
  'jonathan reynolds',
  'jonathan ashworth'],
 'chris hazzard': ['chris law', 'chris heaton-harris', 'christine jardine'],
 'tahir ali': [],
 'jon cruddas': ['john redwood', 'joan ryan'],
 'elisha mccallion': ['lisa cameron', 'michael fallon'],
 'neil hudson': ['nigel huddleston', 'angela richardson', "neil o'brien"],
 'john finucane': ['john mann', 'john nicolson', 'john hayes'],
 'mickey brady': ['patrick grady', 'karen bradley', 'vicky ford'],
 'michelle gildernew': ['michelle donelan', 'michael dugher', 'michael gove'],
 'órfhlaith begley': [],
 'francie molloy': [],
 'tanmanjeetsingh dhesi': ['tanmanjeet singh dhesi'],
 'roger godsiff': ['robert goodwill', 'roger gale']}

In [6]:
mps = mp_tweets['mp_name'].unique()
unmatched = commons_speeches[~(commons_speeches['mp_name'].isin(mps))]['mp_name'].unique()
possible_matches = {x:difflib.get_close_matches(x, mps) for x in unmatched}
possible_matches

{'david nuttall': ['david rutley', 'david mundell', 'david evennett'],
 'rosie winterton': ['beth winter'],
 'fiona mactaggart': ['fiona onasanya'],
 'rob marris': ['rebecca harris', 'carolyn harris', 'robin millar'],
 'speaker': ['steve baker'],
 'eilidh whiteford': ['philippa whitford'],
 'whiteford': ['philippa whitford'],
 'corri wilson': ['chris williamson', 'phil wilson', 'munira wilson'],
 'eagle': ['maria eagle'],
 'clive betts': ['clive lewis', 'clive efford', 'liz saville roberts'],
 'tom elliott': ['julie elliott'],
 'ben howlett': ['ben wallace', 'ben everitt', 'ben lake'],
 'tania mathias': ['tonia antoniazzi'],
 'callum mccaig': [],
 'susan elan jones': ['anew jones', 'sarah jones', 'kevan jones'],
 'ellwood': ['tobias ellwood', 'mike wood'],
 'geoffrey clifton-brown': ['geoffrey cox'],
 'keith vaz': ['valerie vaz'],
 'tasmina ahmed-sheikh': [],
 'gareth johnson': ['caroline johnson', 'joseph johnson', 'gareth snell'],
 'kelvin hopkins': ['rachel hopkins', 'kevin hollinra

In [7]:
manual_matches = {
    'graham jones':'graham p jones',
    'tanmanjeetsingh dhesi': 'tanmanjeet singh dhesi',
    'sirvincecable':'vince cable',
    'marshadecordova':'marsha de cordova',
}

mp_tweets['mp_name'] = mp_tweets['mp_name'].replace(manual_matches)
commons_speeches['mp_name'] = commons_speeches['mp_name'].replace(manual_matches)

In [8]:
unmatched_tweets = mp_tweets[~(mp_tweets['mp_name'].isin(commons_speeches['mp_name'].unique()))]
unmatched_commons = commons_speeches[~(commons_speeches['mp_name'].isin(mp_tweets['mp_name'].unique()))]
print('unmatched tweets count and frac:', len(unmatched_tweets), len(unmatched_tweets)/len(mp_tweets))
print('unmatched commons_speeches count and frac:',len(unmatched_commons), len(unmatched_commons)/len(commons_speeches))


unmatched tweets count and frac: 40735 0.013000458614435472
unmatched commons_speeches count and frac: 35701 0.30240733211357324


In [9]:
mp_tweets.to_parquet('./data/mp_tweets.parquet')
commons_speeches.to_parquet('./data/commons_speeches.parquet')

# add party to mp_tweets

In [1]:
import pandas as pd

In [2]:
commons_speeches = pd.read_parquet('./data/commons_speeches.parquet')
mp_tweets = pd.read_parquet('./data/mp_tweets.parquet')

In [3]:
mp_tweets = mp_tweets.merge(right = commons_speeches[['mp_name','party']].drop_duplicates(), on='mp_name', how='left')


In [5]:
mp_tweets.to_parquet('./data/mp_tweets.parquet')

# add median time to commons_speeches dates

In [6]:
import pandas as pd

In [7]:
commons_speeches = pd.read_parquet('./data/commons_speeches.parquet')

In [9]:
#set time to Noon, as this will be closer than the default (midnight) for estimated time of day
commons_speeches['time']= commons_speeches['time'] + pd.DateOffset(hours=12) 

In [11]:
commons_speeches.to_parquet('./data/commons_speeches.parquet')

In [14]:
commons_speeches['party'].value_counts().index

Index(['Con', 'Lab', 'SNP', 'Lab/Co-op', 'LD', 'DUP', 'Ind', 'PC', 'SDLP',
       'UUP', 'Green', 'Change UK', 'UKIP', 'IGC', 'Alliance'],
      dtype='object')